In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from pointnet.dataset_custom import CustomDataset
from pointnet.model import PointNetDenseCls, feature_transform_regularizer
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import easydict

In [2]:
opt = easydict.EasyDict({'model': '',
                         'batch_size': 4,
                         'nepoch': 50,
                         'workers': 4,
                         'outf': 'seg',
                         'dataset': '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data',
                         'class_choice': 'Head',
                         'feature_transform': True
                        })

In [3]:
dataset = CustomDataset(root = opt.dataset, 
                        classification=False, 
                        class_choice=[opt.class_choice])

dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=opt.batch_size,
                                         shuffle=True,
                                         num_workers=int(opt.workers))

test_dataset = dataset
testdataloader = dataloader

In [4]:
print(len(dataset), len(test_dataset))
num_classes = dataset.num_seg_classes
print('classes', num_classes)

7 7
classes 2


In [5]:
try:
    os.makedirs(opt.outf)
except OSError:
    pass

blue = lambda x: '\033[94m' + x + '\033[0m'

classifier = PointNetDenseCls(k=num_classes, feature_transform=opt.feature_transform)

if opt.model != '':
    classifier.load_state_dict(torch.load(opt.model))

optimizer = optim.Adam(classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
classifier.cuda()

num_batch = len(dataset) / opt.batch_size

In [6]:
for i, data in enumerate(dataloader, 0):
    print(data)
    break

[tensor([[[ 0.3251,  0.1148, -0.3774],
         [-0.4508,  0.0981, -0.0861],
         [ 0.2302,  0.1945, -0.3538],
         ...,
         [ 0.2957,  0.1364, -0.3647],
         [ 0.1589, -0.5739, -0.5490],
         [-0.2575, -0.2002, -0.4415]],

        [[-0.2957, -0.3293, -0.4525],
         [-0.1520,  0.2354, -0.4684],
         [ 0.3153, -0.4257, -0.0979],
         ...,
         [-0.2795, -0.0797,  0.1173],
         [ 0.3072, -0.1062, -0.1866],
         [-0.2795,  0.3891,  0.3672]],

        [[ 0.1769, -0.5575, -0.2466],
         [-0.0084, -0.0269, -0.5778],
         [ 0.3709,  0.3471, -0.3778],
         ...,
         [ 0.6175, -0.1761,  0.0858],
         [ 0.4473, -0.3992,  0.2269],
         [-0.1526, -0.0619, -0.5297]],

        [[-0.3080,  0.5618, -0.0966],
         [-0.2153, -0.0689, -0.1576],
         [-0.0049,  0.5382,  0.3447],
         ...,
         [ 0.2446, -0.3441,  0.0678],
         [-0.0058,  0.3202,  0.5312],
         [-0.3826, -0.3231, -0.2613]]]), tensor([[0, 0, 0,  ...

In [7]:
# my target has -1 somewhere which give cuda assert error

In [8]:
for epoch in range(opt.nepoch):
    scheduler.step()
    for i, data in enumerate(dataloader, 0):
        points, target = data
        points = points.transpose(2, 1)
        points, target = points.cuda(), target.cuda()
        optimizer.zero_grad()
        classifier = classifier.train()
        pred, trans, trans_feat = classifier(points)
        pred = pred.view(-1, num_classes)
        target = target.view(-1, 1)[:, 0] # -1
        #print(pred.size(), target.size())
        loss = F.nll_loss(pred, target)
        if opt.feature_transform:
            loss += feature_transform_regularizer(trans_feat) * 0.001
        loss.backward()
        optimizer.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()
        print('[%d: %d/%d] train loss: %f accuracy: %f' % (epoch, i, num_batch, loss.item(), correct.item()/float(opt.batch_size * 2500)))

        if i % 10 == 0:
            j, data = next(enumerate(testdataloader, 0))
            points, target = data
            points = points.transpose(2, 1)
            points, target = points.cuda(), target.cuda()
            classifier = classifier.eval()
            pred, _, _ = classifier(points)
            pred = pred.view(-1, num_classes)
            target = target.view(-1, 1)[:, 0] # -1
            loss = F.nll_loss(pred, target)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.data).cpu().sum()
            print('[%d: %d/%d] %s loss: %f accuracy: %f' % (epoch, i, num_batch, blue('test'), loss.item(), correct.item()/float(opt.batch_size * 2500)))

    torch.save(classifier.state_dict(), '%s/seg_model_%s_%d.pth' % (opt.outf, opt.class_choice, epoch))

/home/trojan/anaconda3/envs/pointnet/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[0: 0/1] train loss: 0.703526 accuracy: 0.776600
[0: 0/1] test loss: 0.655143 accuracy: 0.891700
[0: 1/1] train loss: 0.683671 accuracy: 0.580000
[1: 0/1] train loss: 0.619542 accuracy: 0.843800
[1: 0/1] test loss: 0.650066 accuracy: 0.888600
[1: 1/1] train loss: 0.623823 accuracy: 0.647700
[2: 0/1] train loss: 0.568924 accuracy: 0.892400
[2: 0/1] test loss: 0.595567 accuracy: 0.894000
[2: 1/1] train loss: 0.606358 accuracy: 0.655200
[3: 0/1] train loss: 0.559753 accuracy: 0.893100
[3: 0/1] test loss: 0.557195 accuracy: 0.894600
[3: 1/1] train loss: 0.510627 accuracy: 0.678500
[4: 0/1] train loss: 0.509494 accuracy: 0.906400
[4: 0/1] test loss: 0.512065 accuracy: 0.905400
[4: 1/1] train loss: 0.508847 accuracy: 0.664900
[5: 0/1] train loss: 0.478209 accuracy: 0.915200
[5: 0/1] test loss: 0.462954 accuracy: 0.906900
[5: 1/1] train loss: 0.507635 accuracy: 0.659300
[6: 0/1] train loss: 0.498200 accuracy: 0.884700
[6: 0/1] test loss: 0.434461 accuracy: 0.887400
[6: 1/1] train loss: 0.4392

In [9]:
## benchmark mIOU
shape_ious = []
for i,data in tqdm(enumerate(testdataloader, 0)):
    points, target = data
    points = points.transpose(2, 1)
    points, target = points.cuda(), target.cuda()
    classifier = classifier.eval()
    pred, _, _ = classifier(points)
    pred_choice = pred.data.max(2)[1]

    pred_np = pred_choice.cpu().data.numpy()
    target_np = target.cpu().data.numpy() - 1

    for shape_idx in range(target_np.shape[0]):
        parts = range(num_classes)#np.unique(target_np[shape_idx])
        part_ious = []
        for part in parts:
            I = np.sum(np.logical_and(pred_np[shape_idx] == part, target_np[shape_idx] == part))
            U = np.sum(np.logical_or(pred_np[shape_idx] == part, target_np[shape_idx] == part))
            if U == 0:
                iou = 1 #If the union of groundtruth and prediction points is empty, then count part IoU as 1
            else:
                iou = I / float(U)
            part_ious.append(iou)
        shape_ious.append(np.mean(part_ious))

print("mIOU for class {}: {}".format(opt.class_choice, np.mean(shape_ious)))

2it [00:00, 10.81it/s]

mIOU for class Head: 0.5503428571428571
